In [9]:
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians


In [10]:
# Download dataset
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

# Read dataset and keep necessary columns
spacex_df = pd.read_csv(spacex_csv_file)
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Get unique launch sites with coordinates
launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_sites_df


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
# NASA Johnson Space Center coordinates (map center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles and labels for each launch site
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    folium.Circle(
        location=coord,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.5,
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

site_map


In [12]:
# Create a function for marker color based on success/failure
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Initialize map (center at NASA)
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add launch records markers to cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Site: {row['Launch Site']}<br>Success: {row['class']}",
        icon=folium.Icon(color=row['marker_color']),
    ).add_to(marker_cluster)

site_map


In [13]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))

    distance = R * c
    return distance


In [14]:
# Coordinates of launch site (e.g., Kennedy Space Center)
launch_site_lat = 28.573255
launch_site_long = -80.646895

# Closest coastline coordinates (example)
coastline_lat = 28.562302
coastline_long = -80.577356

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)
print(f"Distance to coastline: {distance_coastline:.2f} km")

# Initialize map centered on launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_long], zoom_start=10)

# Add launch site marker
folium.Marker(
    [launch_site_lat, launch_site_long],
    popup='Launch Site',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Add coastline marker with distance label
folium.Marker(
    [coastline_lat, coastline_long],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_coastline:.2f} km</b></div>'
    )
).add_to(site_map)

# Draw a line between launch site and coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_long], [coastline_lat, coastline_long]],
    color='green',
    weight=2,
    dash_array='5'
).add_to(site_map)

site_map


Distance to coastline: 6.90 km


In [15]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map
